### Training LSTM model

The following code is designed to train 32 different models by combination of different pre-processing methods. The order of the below processes reflects the order of the preprocessing methods.
<list>
- Expansion of contractions
- Basic preprocessing
- Grammar correction
- Simplification of negations
- Lemmatization
- Removal of stop words

#### Imports

In [3]:
import imports as ii
import functions as f
import preprocessing as pp
import neuralnetworks as nn
import trainRNN as trainRNN

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MUSIC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MUSIC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#### Read-in the datasets

In [4]:
PATH = {}
PATH["dataset_classification"] = "dataset/classification/"
PATH["dataset_labeling"] = "dataset/seq_labeling/"
PATH["music_reviews_train"] = PATH["dataset_classification"] + "music_reviews_train.json.gz"
PATH["music_reviews_dev"] = PATH["dataset_classification"] + "music_reviews_dev.json.gz"
PATH["music_reviews_test"] = PATH["dataset_classification"] + "music_reviews_test.json.gz"
train = f.readJson(PATH["music_reviews_train"])
test = f.readJson(PATH["music_reviews_dev"])

Number of data:  100000
Number of data:  10000


#### Define features of training

Below variables defining:
<list>
- leng = how many sentence will be used for the training
- maxlen = maximum length of words in each sentences both in the training and test data

In [18]:
leng = 30000
maxlen = 70

In [19]:
# read in train and test data

X_train, y_train, train_idx, train_missing_idx = f.json_divide(train)
X_test, y_test, test_idx, test_missing_idx = f.json_divide(test)

# convert labels

sent_dict = {"positive": 1, "negative": 0}
y_train = pp.sentiment_converter(y_train, sent_dict)
y_test = pp.sentiment_converter(y_test, sent_dict)

# sampling

start = 6666 # just a number

X_train = X_train[start:start+leng]
y_train = y_train[start:start+leng]


#### Grid search & Training
Below code generating 32 combinations of each preprocessing methods on the train and test set. The structure of the generated datasets is:
<list> 
- data_sets[0] - First preprocessing combination
- data_set[0][0] - Labels array of the combinations. For instance:  [0,1,0,0,0,0] meaning only the basic pre-processing was applied, the rest was not
- data_sets[0][1] - Preprocessed Train set
- datasets[0][2] - Preprocessed Dev set
    
</list>

It's running for a while since it is generating a relatively huge set

In [10]:
data_sets, y_train, y_test = f.grid_search(X_train, X_test, y_train, y_test)

#### Ratio of fully catched sentences
Basic statistics about the ratio of sentencies which were fully catched with the given maximum sentence length variable

In [ ]:
f.statistics_sets_sizes(data_sets, max_len=maxlen)

### Training the LSTM model
Training and evaulation takes around one hour with 5000 sentences

In [ ]:
trainRNN.trainRNN(data_sets, y_train, y_test, early_stop_patience=2, filename="dummy", maxlen=maxlen)

### Re-training an LSTM model
The code below is written for train a model with a given pre-process combination or combinations. Note: Early stopping criteria set to 10, meaning model is able to overfit.

In [ ]:
# define combinations
combination = [[0, 1, 0, 0, 1, 0],
[0, 1, 1, 0, 1, 0],
[0, 1, 1, 1, 1, 0],
[0, 1, 0, 1, 1, 0],
[1, 1, 1, 1, 1, 0],
[1, 1, 0, 0, 1, 0],
[1, 1, 1, 0, 1, 0],
[1, 1, 0, 1, 1, 0],
[0, 1, 0, 1, 1, 1],
[0, 1, 0, 0, 1, 1],
[1, 1, 0, 0, 1, 1],
[1, 1, 0, 1, 1, 1],
[0, 1, 1, 1, 1, 1],
[0, 1, 1, 0, 1, 1],
[1, 1, 1, 0, 1, 1],
[1, 1, 1, 1, 1, 1],
]

# name of the file which will contain the LOGS
filename = "30k/retraining"

# early stopping criteria - max 10! 
early_stop_patience = 2

for comb in combination:
    data_sets, y_train, y_test = f.grid_search_retrain(X_train, X_test, y_train, y_test, comb)
    trainRNN.trainRNN(data_sets, y_train, y_test, early_stop_patience=early_stop_patience, filename=filename, maxlen=maxlen)

Combinations:  [0, 1, 0, 0, 1, 0]
Epoch 1/10
 681/1200 [================>.............] - ETA: 42s - loss: 8.9551 - accuracy: 0.4194